In [1]:
from requests_html import HTMLSession
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
from lxml import html
import numpy as np
import re

session=HTMLSession()

In [2]:
def case_split_cpu(CPU):
    if (CPU[0].find('Model:')!=-1):
        name_cpu=np.nan
        type_cpu=np.nan
    elif (CPU[0].find('GHz')!=-1): 
        name_cpu=np.nan
        type_cpu=np.nan
    elif (CPU[1].find('GHz')!=-1):
        name_cpu=CPU[0].rsplit(' ',1)[0]
        type_cpu=CPU[0].rsplit(' ',1)[1]
    elif (CPU[1].find('Intel')!=-1):
        name_cpu=CPU[0].rsplit('-',1)[0]
        type_cpu=CPU[0].rsplit('-',1)[1]
    else:
        name_cpu=CPU[0]
        type_cpu=CPU[1]
    return name_cpu,type_cpu

In [3]:
def Get_URL_List(baseurl):
    list_url=[]
    list_url.append(baseurl)
    while True:
        r = session.get(baseurl)
        soup = BeautifulSoup(r.content, 'html.parser')
        url = soup.find("a", {"class": "btn_next ty-pagination__item ty-pagination__btn ty-pagination__next cm-history"})
        if (url is None):
            break
        else:
            next_link=url.attrs['href']
            baseurl=next_link
            list_url.append(next_link)
    return list_url

In [4]:
def Get_Page(baseurl):
    list_url=Get_URL_List(baseurl)
    list_page=[]
    for i in list_url:
        url=i
        r = session.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')
        url = soup.findAll("a", {"class": "nk-link-product"})    
        for i in url:
            page=i.attrs['href']
            list_page.append(page)
    return list_page

In [5]:
def Crawl_data(baseurl,Y):
    list_page=Get_Page(baseurl)
    title_l=[]
    price_l=[]
    name_cpu_l=[]
    type_cpu_l=[]
    ram_l=[]
    type_ram_l=[]
    storage_l=[]
    type_storage_l=[]
    graphic=[]
    brand_l=[]
    star_l=[]
    vote_l=[]
    weight_l=[]
    type_laptop=[]
    for i in range(len(list_page)):


        r=session.get(list_page[i])
        title=r.html.find('h1',first=True).text
        #print('Name: ',title)
        title_l.append(title)
        price_t=r.html.find('.nk-price-final',first=True).text
        price=price_t.split('đ')[0]
        price=price.replace('.','')
        #print('Price: ',price)
        price_l.append(price)
        star=r.html.find('.number_avg_rate_npv',first=True).text
        vote=r.html.find('.total_vote',first=True).text.split(' ')[1]
        detail=r.html.find('.popup',first=True).text.split('\n')


        CPU=[]


        storage=np.nan
        ram=np.nan
        type_ram=np.nan
        type_storage=''
        chipset_graphic=np.nan
        weight=np.nan


        for j in range(len(detail)):
            if (detail[j].find('CPU')!=-1):
                CPU.append(detail[j+1])
            if (detail[j].find('Dung lượng RAM:')!=-1):
                ram=detail[j+1]
            if (detail[j].find('Loại RAM:')!=-1):
                type_ram=detail[j+1]
            if (detail[j].find('Dung lượng :')!=-1):
                storage=detail[j+1]
            if (detail[j].find('Loại ổ đĩa cứng:')!=-1):
                type_storage=detail[j+1]
            if (detail[j].find('Chipset card đồ họa:')!=-1):
                chipset_graphic=detail[j+1]
            elif (detail[j].find('Bộ xử lý đồ họa:')!=-1):
                chipset_graphic=detail[j+1]
            if (detail[j].find('Khối lượng')!=-1):
                weight=detail[j+1]

        if(title.split(' ')[0]!='Laptop'):
            brand=title.split(' ')[0]
        if(title.split(' ')[0]!='LAPTOP'):
            brand=title.split(' ')[0]
        if(title.split(' ')[0]=='Laptop'):
            brand=title.split(' ')[1]
        if(title.split(' ')[0]=='LAPTOP'):
            brand=title.split(' ')[1]


        


        vote_l.append(vote)
        star_l.append(star)
        ram_l.append(ram)
        type_ram_l.append(type_ram)
        storage_l.append(storage)
        type_storage_l.append(type_storage)
        graphic.append(chipset_graphic)
        name_cpu,type_cpu=case_split_cpu(CPU)
        name_cpu_l.append(name_cpu)
        type_cpu_l.append(type_cpu)
        brand_l.append(brand)
        weight_l.append(weight)
        type_laptop.append(Y)


        
    data={'Name':title_l,'Price':price_l,'CPU':name_cpu_l,'Type_CPU':type_cpu_l,'Storage':storage_l,
        'Type_Storage':type_storage_l,'RAM':ram_l,'Type_RAM':type_ram_l,'Graphic':graphic,'Weight':weight_l,
        'Score_star':star_l,'Vote':vote_l,'Brand':brand_l,'Type_laptop':type_laptop}
    df=pd.DataFrame(data)
    return df

In [6]:
df_gaming=Crawl_data('https://www.nguyenkim.com/may-tinh-xach-tay/?sort_by=position&sort_order=desc&features_hash=367-97743','laptop-gaming')
df_dohoa=Crawl_data('https://www.nguyenkim.com/may-tinh-xach-tay/?sort_by=position&sort_order=desc&features_hash=371-97747','do-hoa-ky-thuat')
df_vanphong=Crawl_data('https://www.nguyenkim.com/may-tinh-xach-tay/?sort_by=position&sort_order=desc&features_hash=373-97749','hoc-tap-van-phong')

In [7]:
frames = [df_gaming, df_dohoa, df_vanphong]
df=pd.concat(frames)

In [8]:
df

,Name,Price,CPU,Type_CPU,Storage,Type_Storage,RAM,Type_RAM,Graphic,Weight,Score_star,Vote,Brand,Type_laptop
0,Laptop Dell Vostro 3580 (T3RMD2),20990000,Intel Core i7,8565U,256GB,SSD,8 GB,DDR4,AMD Radeon 520,2.21 kg,0,0,Dell,laptop-gaming
1,Laptop MSI GF75 9RCX-430VN,23490000,Intel Core i7,9750H,256 GB,SSD,8 GB,DDR4,GeForce GTX 1050Ti 4GB,2.2 kg,0,0,MSI,laptop-gaming
2,LAPTOP ASUS S430FA-EB074T VÀNG,15390000,Intel Core i5,8265U,1TB,HDD,4 GB,DDR4,Intel UHD Graphics 620,1.4 kg,0,0,ASUS,laptop-gaming
3,Laptop MSI GF63 9RCX - 646VN,20990000,Intel Core i5,9300H,NaN,NVMe PCIe SSD M2 - 2280,8 GB,DDR4,NVIDIA GeForce GTX 1050Ti 4GB GDDR5 + Intel UH...,2.3 kg,4.5,10,MSI,laptop-gaming
4,LAPTOP ACER ASPIRE E5-576G-88EP,15990000,Intel Core i7,8550U,NaN,"HDD: 1 TB, Intel Optane: 16GB",4 GB,DDR3L,NVIDIA GeForce MX130,2.23 kg,0,0,ACER,laptop-gaming
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Macbook Pro 16.0 inch 512GB Silver (MVVL2SA/A),59990000,Intel Core,i7,512 GB,SSD,16 GB,DDR4,NaN,2.0 kg,0,0,Macbook,hoc-tap-van-phong
168,Macbook Pro 16.0 inch 512GB Gray (MVVJ2SA/A),59990000,Intel Core,i7,512 GB,SSD,16 GB,DDR4,NaN,2.0 kg,0,0,Macbook,hoc-tap-van-phong
169,Macbook Pro 16.0 inch 512GB Silver (MVVL2SA/A),59990000,Intel Core,i7,512 GB,SSD,16 GB,DDR4,NaN,2.0 kg,0,0,Macbook,hoc-tap-van-phong
170,Macbook Pro 16.0 inch 1TB Gray (MVVK2SA/A),69990000,Intel Core,i9,1 TB,SSD,16 GB,DDR4,NaN,2.0 kg,0,0,Macbook,hoc-tap-van-phong


In [9]:
df.to_csv('NguyenKim.csv',index=0)